# Goals

Good data science projects start with a clear understanding of the data and where it comes from. In this tutorial, we will use HTTP requests and BeautifulSoup to write a web scraper, and then we will use NumPy, Pandas, matplotlib, sklearn, and Jupyter notebooks to explore that data carefully.

The goal of this part of the tutorial is build your skills in: 

- asking good questions
- finding good data
- scraping a web page
- cleaning and manipulating data
- using pandas
- exploratory data visualization
- thinking about data and what features might be used to make predictions

## We're going to make a movie! 

Our first goal today is to learn about what makes a box office hit. So to start off let's pretend that we are some (very data savvy) movie producers, and we want to make a movie that will make us a metric ton of money. So what are some features of movies that might correlate with making a ton of money? Is it the budget? The actors? 

##### Example 1.  Black Panther may be a contender for best movie EVARR

<img src="http://www.nerdcoremovement.com/wp-content/uploads/Black-Panther-1-938x535.jpg"
     alt="Example Great Movie"
     style="float: left; margin-right: 10px;"/>
    

### Try it out!  

List some of the features that you think you might want to consider as you make you make **The Best Movie Ever**.

*type your answer here*

# Asking good questions  

**"A good question is one that you can answer!"**

Many of the questions that we want to ask are causal (e.g. "Will this script be a box office hit?"), but causality can be difficult to prove. You can read more about some ways to argue for causality. 

In the exploratory data analysis phase, most of the questions you will be able to answer will be of a different 

@help can someone come up with some exercises for this? Maybe have them rewrite some causal statements as correlational?

*type your answer here*

# Getting good data 

## What are some good places to get data? 

There are a lot of good places to get data! If you want to start looking at some data sets, here are a few repositories. 
   - [Kaggle](https://www.kaggle.com/datasets)  
   - [FiveThirtyEight](https://data.fivethirtyeight.com/)
   - [Government Data](https://catalog.data.gov/dataset)
   - APIs (e.g. [IMDBPy](https://imdbpy.sourceforge.io/))
   - Scraping the Internet! 
    
### What sites should you scrape? 

When you want to find a certain type of data, try to find data that is structured (e.g. Wikipedia). Tables, headings, and text boxes stand out, and can be easier to find. **Also, remember to read the Terms of Service for the site that you plan to scrape.**

### Try it out! 

We are going to use data to make **The Best Movie Ever**, but what are some good places for us to get that data? 

*type your answer here*

## Start scraping 

To get started, we need to import several packages. We explain what several of these imports are below in the comments. 

In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline 
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
from matplotlib import rcParams # special matplotlib argument for improved plots
from collections import defaultdict 
#from imdb import IMDb

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import cPickle as pickle
import seaborn as sns
# sns.set_style("whitegrid")
# sns.set_context("poster")

import io 
import time
import requests
import sklearn
import warnings
warnings.filterwarnings('ignore')

ImportError: No module named seaborn

We use [Seaborn](http://seaborn.pydata.org/) to give us a nicer default color palette, with our plots being of large (poster) size and with a white-grid background. 

### Scrape Box Office Mojo 

To get the text from the website to your local machine, we will use a GET request, which is available in the [Requests](http://docs.python-requests.org/en/master/) library. 

To get started exploring the text that you bring to your local macine, we will use [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/). If you are familiar with another scraping library like [PyQuery](https://pythonhosted.org/pyquery/) or [Scrapy](https://scrapy.org/), feel free to do this exercise using those instead (or in addition! :)). 

In [3]:
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
import time

Here, we access a webpage and download the HTML using requests. When we make a GET response, we get an HTTP response object back. 

In [4]:
r_2018 = requests.get("http://www.boxofficemojo.com/yearly/chart/?view=releasedate&view2=domestic&page=1&yr=2018")

You should get a HTTP response 200, which means that the request went through without issue. If you get another HTTP response, you can look it up in [this list](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) to determine what it is. 

Alternatively, if you like your HTTP status codes illustrated as cat gifs, you can look up your codes using [http.cat](https://http.cat/).

In [5]:
print r_2018

<Response [200]>


There are a lot of awesome things going in this response object. Most relevantly, it has returned all the text from the page that we made the request from to us, so we can look at it on our local machine. 

In [6]:
## uncomment the line below to see the html returned by the response 
#print r_2018.text

While this blob of text is not difficult for our computer to search through, it can be a little difficult for us to wrap human brains around. Another way that we can look at the text are: 
(a) Right click > View Source. 

@help insert screen shot 

(b) Right click > Inspect Item 

@help insert screen shot 

(c) View > Developer > Developer Tools 

@help insert screen shot 

### Try it out! 

### Understanding the HTML 

Which parts of this text do we need to get information about these movies? How can we pull out only the information that we want? 

*double-click to type your answer here*

### Try it out!

### Requests 

Scrape Box Office Mojo's data for movies from 2016 to 2018 and store the page text in a dictionary. Remember to use the time(s) function to add a wait before each GET request. 

In [7]:
# Your answer here
years = xrange(2016,2018)
pages = xrange(1,9)
year_pagetxt = {}
for year in years: 
    pagestext = {}
    for page in pages: 
        r = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=%s&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page, year))
        pagestext[page] = r.text
        time.sleep(1)
    year_pagetxt[year] = pagestext

### Try it out! 

### Beautiful Soup 

Store the information from the table in the dictionary with the column heading as the key and a list of values as the value. 

In [8]:
# Your answer here 

# This loop cycles through the data obtained on Box Office Mojo
## And placed in a dictionary for recall later when folding in IMDb information
movie_budget = defaultdict(list) 
for year in years: 
    for page in pages: 
        soup = BeautifulSoup(year_pagetxt[year][page], "html.parser")
        rows = soup.find_all("font", attrs={'size':'2'})
           
        start = 10 
        for i in range(start,len(rows)-2):
            t = rows[i].get_text()
            if unicode('Summary of') in t: 
                break
            elif (i-start) % 9 == 0: 
                movie_budget['rank'].append(t)
            elif (i-start) % 9 == 1: 
                movie_budget['year'].append(year)
                r = '('+str(year)
                if unicode(r) in t: 
                    j = t.index(unicode(r))
                    movie_budget['title'].append(t[:j])
                else: 
                    movie_budget['title'].append(t)
            elif (i-start) % 9 == 2: 
                movie_budget['studio'].append(t)
            elif (i-start) % 9 == 3: 
                movie_budget['gross'].append(t)
            elif (i-start) % 9 == 4: 
                movie_budget['gross theaters'].append(t)
            elif (i-start) % 9 == 5: 
                movie_budget['opening'].append(t)
            elif (i-start) % 9 == 6: 
                movie_budget['opening theaters'].append(t)
            elif (i-start) % 9 == 7: 
                movie_budget['open'].append(t)
            elif (i-start) % 9 == 8: 
                movie_budget['close'].append(t)

# Exploratory Data Analysis Preview 

Before we do any fancy statistical modeling, we need to understand our data. To do this, there is no substitute for looking at our data, making some scatter plots, and getting a better idea of what it looks like. 

### Explain the data 

@help explain the data and what is limiteddf 


### Some Features to Explore 

- **Movie Budget**
- **Opening Weekend**
- **Opening Theaters**
- **IMDb rating**
- **Seasonality**
- **MPAA Rating**
- **Power Studios**

## Example correlation plots 

Remember, the goal of all of this data analysis is to make a ton of money, so it might make sense to see how each of these factors correlate with **gross revenue**. 

### Linear regression 

One nice way of exploring how two variables look in relationship to one another is to make a regression plot. A nice explanation of linear regression can be found [here](https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html). Most data analysis tools have some way of doing linear regression. In this tutorial, we will use the [seaborn regplot](https://seaborn.pydata.org/tutorial/regression.html) to plot regressions.  

#### Gross Revenue vs. Opening Weekend Revenue

In [11]:
sns.regplot(y="gross", x="opening_gross", data=limiteddf, fit_reg = True)

NameError: name 'sns' is not defined

#### Gross Revenue vs. Opening Weekend Revenue

In [ ]:
sns.regplot(y="gross", x="opening_theaters", data=limiteddf, fit_reg = True)

#### Gross Revenue vs Allocated Budget 

In [ ]:
sns.regplot(y="gross", x="budget", data=limiteddf, fit_reg = True)

#### Gross revenue vs IMDB rating 

In [12]:
sns.regplot(y="gross", x="rating", data=limiteddf, fit_reg = True)

NameError: name 'sns' is not defined

### Bonus: What other features? 

There is always more to explore and discover in the data. Is there anything else that you wish that we had explored together? If so, feel free to try it out now! 

If you find yourself feeling stuck, you can check out [oscarpredictor.github.io](http://oscarpredictor.github.io/) for inspiration. 